## 4PopSplit  

In [1]:
import stdpopsim
import numpy as np
from matplotlib import pyplot as plt
import statistics
import math
import tskit
from collections import Counter
import csv
from sklearn.linear_model import LinearRegression
import msprime

In [14]:
def split(N_A, N_B, N_C, N_D, split_time1, split_time2):

    # Times are provided in years, so we convert into generations.
    generation_time = 25
    T_S1 = split_time1 / generation_time
    T_S2 = split_time2 / generation_time


    # Population IDs correspond to their indexes in the population
    # configuration array. Therefore, we have 0=A, 1=B, 2=C, 3=D initially.
    population_configurations = [
        msprime.PopulationConfiguration(
            sample_size=500, initial_size=N_A),
        msprime.PopulationConfiguration(
            sample_size=500, initial_size=N_B), 
        msprime.PopulationConfiguration(
            sample_size=500, initial_size=N_C),
        msprime.PopulationConfiguration(
            sample_size=500, initial_size=N_D)
    ]

    demographic_events = [
        msprime.MassMigration(
            time=T_S2, source=3, destination=2, proportion=1.0),
        msprime.MassMigration(
            time=T_S2, source=1, destination=0, proportion=1.0),
        msprime.MassMigration(
            time=T_S1, source=2, destination = 0, proportion=1.0)
    ]
    # Use the demography debugger to print out the demographic history
    # that we have just described.
    dd = msprime.DemographyDebugger(
        population_configurations=population_configurations,
        demographic_events=demographic_events)
    #dd.print_history()
    
    ts = msprime.simulate(population_configurations=population_configurations,
                         demographic_events=demographic_events, length=1e6, recombination_rate=1e-8,
                         mutation_rate=1e-8)
    return ts

In [8]:
def genotype_matrix(ts):
    G = ts.genotype_matrix()
    A = G[:,0:500]
    B = G[:,500:1000]
    C = G[:,1000:1500]
    D = G[:,1500:2000]
    return A, B, C, D

In [20]:
def make_panel(pop1, pop2):
    return np.concatenate((pop1,pop2), axis=0)

In [28]:
def make_test_vec():
    t = np.zeros(1000)
    t[0:500] = 1
    return t

In [47]:
def make_TXM(T,X,M):
    tmp = np.dot(T, X)
    return np.dot(tmp,np.transpose(M))

In [48]:
def make_MXT(T,X,M):
    tmp = np.dot(M,np.transpose(X))
    return np.dot(tmp,np.transpose(T))

In [54]:
ts = split(2000,2000,2000,2000,140e3,70e3)
geno_mat = genotype_matrix(ts)
A = np.transpose(geno_mat[0])
B = np.transpose(geno_mat[1])
C = np.transpose(geno_mat[2])
D = np.transpose(geno_mat[3])
X = make_panel(B,D)
M = make_panel(A,C)
T = make_test_vec()

txm = make_TXM(T,X,M)
mxt = make_MXT(T,X,M)

final = np.dot(txm,mxt)
final 

653913305051.0

In [57]:
#np.savetxt("../data/4PopSplit/testBD_X.txt", X)
np.savetxt("../data/4PopSplit/testBD_M.txt", M)
np.savetxt("../data/4PopSplit/testBD_T.txt", T)

In [58]:
ts = split(2000,2000,2000,2000,140e3,70e3)
geno_mat = genotype_matrix(ts)
A = np.transpose(geno_mat[0])
B = np.transpose(geno_mat[1])
C = np.transpose(geno_mat[2])
D = np.transpose(geno_mat[3])
X = make_panel(C,D)
M = make_panel(A,B)
T = make_test_vec()

txm = make_TXM(T,X,M)
mxt = make_MXT(T,X,M)

final = np.dot(txm,mxt)
final 

75019944726.0

In [59]:
np.savetxt("../data/4PopSplit/testCD_X.txt", X)
np.savetxt("../data/4PopSplit/testCD_M.txt", M)
np.savetxt("../data/4PopSplit/testCD_T.txt", T)